In [9]:
import numpy as np
import pandas as pd
import requests
import json
import nba_api
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonallplayers
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.live.nba.endpoints import boxscore
import time
import sqlite3


## Players

In [10]:
players = commonallplayers.CommonAllPlayers()
players_df = players.get_data_frames()[0]
players_df = players_df.query("TO_YEAR=='2025'").reset_index(drop=True)
players_df.columns = [x.lower() for x in players_df.columns]
players_df.to_csv('players.csv', index=False) 

## Teams

In [ ]:
nba_teams = teams.get_teams()
teams_df = pd.DataFrame(nba_teams)
teams_df = teams_df.rename({'id': 'team_id'}, axis=1) 
teams_df.to_csv('teams.csv', index=False) 



## Team games

In [13]:
teamid = teams_df['team_id']

def getteamdata(t):
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=t)
    games = gamefinder.get_data_frames()[0]
    games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
    games = games[(games['GAME_DATE'] >= pd.to_datetime('10-22-2024')) & 
                  (games['GAME_DATE'] <= pd.to_datetime('4-13-2025'))]
    return games

team_data = []
for t in teamid:
    print(t)
    time.sleep(10)
    team_data.append(getteamdata(t))

teamgame_df = pd.concat(team_data, ignore_index=True)
teamgame_df.columns = [x.lower() for x in teamgame_df.columns]
teamgame_df.to_csv('teamgames.csv', index=False) 

1610612737


KeyboardInterrupt: 

## Games and Player game
BoxScoreTraditionalV2

In [ ]:
teamgame_df.query("team_name=='Atlanta Hawks'")

NameError: name 'teamgame_df' is not defined

In [ ]:
gameids = teamgame_df['game_id'].unique()

def gamedata(g):

    box = boxscore.BoxScore(g) 
    game =box.game.get_dict()
    game_dict = {}
    game_dict['gameId'] = game['gameId']
    game_dict['homeTeam'] = game['homeTeam']['teamId']
    game_dict['awayTeam'] = game['awayTeam']['teamId']
    game_dict['attendance'] = game['attendance']
    game_dict['sellout'] = game['sellout']
    game_dict['duration'] = game['duration']
    game_dict['regulationPeriods'] = game['regulationPeriods']

    return game_dict, box


In [ ]:
def playergame_home(box):
        playergame_df =pd.json_normalize(box.home_team_player_stats.get_dict())
        playergame_df.columns = [x.lower().replace('statistics.', '') for x in playergame_df.columns]

        todrop = ['order', 'jerseynum', 'oncourt', 'played', 'name', 'namei', 'firstname', 'familyname',
                'blocksreceived', 'fieldgoalspercentage', 'freethrowspercentage', 'minus', 'plus', 
                'minutescalculated', 'threepointerspercentage', 'twopointersattempted', 'twopointersmade',
                'twopointerspercentage']
        playergame_df = playergame_df.drop(todrop, axis=1)
        playergame_df['minutes']= [int(x[2:4])+int(x[5:7])/60 for x in playergame_df['minutes']]
        playergame_df['HA'] = 'Home'
        return playergame_df

def playergame_away(box):
        playergame_df =pd.json_normalize(box.away_team_player_stats.get_dict())
        playergame_df.columns = [x.lower().replace('statistics.', '') for x in playergame_df.columns]

        todrop = ['order', 'jerseynum', 'oncourt', 'played', 'name', 'namei', 'firstname', 'familyname',
                'blocksreceived', 'fieldgoalspercentage', 'freethrowspercentage', 'minus', 'plus', 
                'minutescalculated', 'threepointerspercentage', 'twopointersattempted', 'twopointersmade',
                'twopointerspercentage']
        playergame_df = playergame_df.drop(todrop, axis=1)
        playergame_df['minutes']= [int(x[2:4])+int(x[5:7])/60 for x in playergame_df['minutes']]
        playergame_df['HA'] = 'Away'
        return playergame_df

In [ ]:
games = []
playergames = []
for g in gameids:
    print(g)
    time.sleep(1)
    gm, box = gamedata(g)
    games.append(gm)

    pg = playergame_home(box)
    pg['gameid'] = g
    playergames.append(pg)

    pg = playergame_away(box)
    pg['gameid'] = g
    playergames.append(pg)
    
games_df = pd.DataFrame(games)
games_df.to_csv('games.csv', index=False) 

playergame_df = pd.concat(playergames, ignore_index=True)
playergame_df.to_csv('playergames.csv', index=False)

0022401186
0022401173
0022401169
0022401149
0022401136
0022401128
0022401109
0022401097
0022401083
0022401062
0022401049
0022401031
0022401025
0022400993
0022400978
0022400960
0022400945
0022400928
0022400914
0022400899
0022400884
0022400878
0022400851
0022400841
0022400825
0022400814
0022400790
0022400773
0022400756
0022400744
0022400736
0022400719
0022400701
0022400686
0022400675
0022400532
0022400656
0022400639
0022400623
0022400612
0022400602
0022400587
0022400563
0022400556
0022400522
0022400506
0022400486
0022400477
0022400461
0022400438
0022400427
0022400413
0022400395
0022400378
0022400370
0022401229
0022401202
0022400350
0022400334
0022400323
0022400315
0022400300
0022400041
0022400287
0022400280
0022400030
0022400258
0022400250
0022400239
0022400012
0022400001
0022400198
0022400185
0022400171
0022400157
0022400152
0022400135
0022400121
0022400103
0022400100
0022400079
0022400064
0022401187
0022401174
0022401156
0022401151
0022401137
0022401120
0022401106
0022401093
0022401080

ReadTimeout: HTTPSConnectionPool(host='cdn.nba.com', port=443): Read timed out. (read timeout=30)

## Get into 3NF

In [ ]:
playergame_df = pd.read_csv('playergames.csv')
games_df = pd.read_csv('games.csv')
teams_df = pd.read_csv('teams.csv')
players_df = pd.read_csv('players.csv')
teamgame_df = pd.read_csv('teamgames.csv')

games_df = games_df.drop(['homeTeam', 'awayTeam'], axis=1)
players_df = players_df.drop(['team_city', 'team_name', 'team_abbreviation', 'team_code',
       'team_slug'], axis=1)

In [ ]:
teamgame_df['HA'] = teamgame_df['matchup'].apply(lambda x: 'Home' if 'vs.' in x else 'Away')
teamgame_df = teamgame_df.rename({'game_id': 'gameid'}, axis=1)


In [ ]:
playergame_df = pd.merge(playergame_df, teamgame_df[['gameid', 'HA', 'team_id']], 
                         how='left', 
                         on=['gameid', 'HA'])
playergame_df = playergame_df.drop('HA', axis=1)

In [ ]:
players_df = players_df.rename({'person_id': 'personid', 'team_id': 'teamid'}, axis=1)
games_df = games_df.rename({'gameId': 'gameid'}, axis=1)
teams_df = teams_df.rename({'team_id': 'teamid'}, axis=1)
teamgame_df = teamgame_df.rename({'team_id': 'teamid'}, axis=1)

## Build DB

In [ ]:
nbadb = sqlite3.connect('nba2024.db')

games_df.to_sql('games', nbadb, index=False, if_exists='replace')

playergame_df.to_sql('playergames', nbadb, index=False, if_exists='replace')

players_df.to_sql('players', nbadb, index=False, if_exists='replace')

teamgame_df.to_sql('teamgames', nbadb, index=False, if_exists='replace')

teams_df.to_sql('teams', nbadb, index=False, if_exists='replace')

30

## Document the DB

In [ ]:
def pandas_df_to_dbml(df: pd.DataFrame, table_name: str) -> str:
    """
    Converts a pandas DataFrame to a DBML string.

    Args:
        df: The pandas DataFrame to convert.
        table_name: The name of the table in the DBML schema.

    Returns:
        A DBML string representing the DataFrame schema.
    """

    dbml_string = f"Table {table_name} {{\n"

    for column_name, column_type in df.dtypes.items():
        dbml_type = map_pandas_dtype_to_dbml_type(column_type)
        dbml_string += f"  {column_name} {dbml_type}\n"

    dbml_string += "}\n"
    return dbml_string

def map_pandas_dtype_to_dbml_type(dtype) -> str:
    """Maps a pandas dtype to a DBML type."""
    dtype_name = str(dtype)
    if "int" in dtype_name:
      return "int"
    if "float" in dtype_name:
      return "float"
    if "datetime" in dtype_name:
        return "datetime"
    return "varchar"

In [ ]:
print(pandas_df_to_dbml(teams_df, 'teams'))

Table teams {
  teamid int
  full_name varchar
  abbreviation varchar
  nickname varchar
  city varchar
  state varchar
  year_founded int
}



In [ ]:
print(pandas_df_to_dbml(games_df, 'games'))

Table games {
  gameid int
  attendance int
  sellout int
  duration int
  regulationPeriods int
}



In [ ]:
print(pandas_df_to_dbml(playergame_df, 'playergames'))

Table playergames {
  status varchar
  personid int
  position varchar
  starter int
  assists int
  blocks int
  fieldgoalsattempted int
  fieldgoalsmade int
  foulsoffensive int
  foulsdrawn int
  foulspersonal int
  foulstechnical int
  freethrowsattempted int
  freethrowsmade int
  minutes float
  plusminuspoints float
  points int
  pointsfastbreak int
  pointsinthepaint int
  pointssecondchance int
  reboundsdefensive int
  reboundsoffensive int
  reboundstotal int
  steals int
  threepointersattempted int
  threepointersmade int
  turnovers int
  notplayingreason varchar
  notplayingdescription varchar
  gameid int
  team_id float
}



In [ ]:
print(pandas_df_to_dbml(players_df, 'players'))

Table players {
  personid int
  display_last_comma_first varchar
  display_first_last varchar
  rosterstatus int
  from_year int
  to_year int
  playercode varchar
  player_slug varchar
  teamid int
  games_played_flag varchar
  otherleague_experience_ch int
}



In [ ]:
print(pandas_df_to_dbml(teamgame_df, 'teamgames'))

Table teamgames {
  season_id int
  teamid int
  team_abbreviation varchar
  team_name varchar
  gameid int
  game_date varchar
  matchup varchar
  wl varchar
  min int
  pts int
  fgm int
  fga int
  fg_pct float
  fg3m int
  fg3a float
  fg3_pct float
  ftm int
  fta int
  ft_pct float
  oreb float
  dreb float
  reb float
  ast int
  stl float
  blk int
  tov int
  pf int
  plus_minus float
  HA varchar
}



In [ ]:
teams_df

,teamid,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [ ]:
games_df

,gameid,attendance,sellout,duration,regulationPeriods
0,22401186,17714,1,114,4
1,22401173,19752,1,129,4
2,22401169,17926,1,136,4
3,22401149,18846,1,142,4
4,22401136,16331,0,133,4
...,...,...,...,...,...
1226,22400237,14789,0,128,4
1227,22400752,18740,0,136,4
1228,22400469,18866,0,143,4
1229,22400260,14671,0,160,4


In [ ]:
myquery = '''
SELECT *
FROM games
INNER JOIN teamgames
    ON games.gameid = teamgames.gameid
'''
pd.read_sql_query(myquery, con=nbadb)

,gameid,attendance,sellout,duration,regulationPeriods,season_id,teamid,team_abbreviation,team_name,gameid,...,oreb,dreb,reb,ast,stl,blk,tov,pf,plus_minus,HA
0,22401186,17714,1,114,4,22024,1610612737,ATL,Atlanta Hawks,22401186,...,9.0,35.0,44.0,32,8.0,2,15,15,12.0,Home
1,22401186,17714,1,114,4,22024,1610612753,ORL,Orlando Magic,22401186,...,12.0,31.0,43.0,25,11.0,5,14,8,-12.0,Away
2,22401173,19752,1,129,4,22024,1610612737,ATL,Atlanta Hawks,22401173,...,18.0,35.0,53.0,28,9.0,5,14,15,14.0,Away
3,22401173,19752,1,129,4,22024,1610612755,PHI,Philadelphia 76ers,22401173,...,14.0,31.0,45.0,29,6.0,6,14,17,-14.0,Home
4,22401169,17926,1,136,4,22024,1610612737,ATL,Atlanta Hawks,22401169,...,7.0,44.0,51.0,36,11.0,1,15,19,24.0,Away
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,22400469,18866,0,143,4,22024,1610612766,CHA,Charlotte Hornets,22400469,...,9.0,36.0,45.0,25,8.0,6,17,10,-4.0,Away
2458,22400260,14671,0,160,4,22024,1610612765,DET,Detroit Pistons,22400260,...,21.0,35.0,56.0,30,8.0,3,19,23,-2.0,Away
2459,22400260,14671,0,160,4,22024,1610612766,CHA,Charlotte Hornets,22400260,...,11.0,32.0,43.0,23,12.0,5,15,22,2.0,Home
2460,22400169,13247,0,132,4,22024,1610612765,DET,Detroit Pistons,22400169,...,6.0,40.0,46.0,34,5.0,4,10,20,-1.0,Away
